In [1]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import pickle
import pandas as pd
from tqdm import tqdm
import seaborn as sns

from graspy.plot import heatmap

from mgcpy.independence_tests.dcorr import DCorr
from mgcpy.independence_tests.rv_corr import RVCorr
from mgcpy.independence_tests.mgc import MGC

from simulations import rho_ER_marg, rho_sbm_marg, rho_sbm_diff_block
from utils import triu_no_diag, to_distance_mtx, identity, sbm_params, estimate_block_assignment, block_permute, sort_graph, pearson_power

In [2]:
def dcorr_power_two_sided(indept_test, sim_func, mc=500, alpha=0.05, given_blocks=False, blocks=None, **kwargs):
    # power for any test that builds on distance matrices
    # can use dcorr / mgc
    test_stat_null_array = np.zeros(mc)
    test_stat_alt_array = np.zeros(mc)
    for i in range(mc):
        A, B = sim_func(**kwargs)
        if given_blocks:
            block_assignment = blocks
        else:
            block_assignment = estimate_block_assignment(A, B)
        A_null = block_permute(A, block_assignment)
        B_sorted = sort_graph(B, block_assignment)
        
        test_stat_alt, _ = indept_test.test_statistic(
            matrix_X=to_distance_mtx(A), matrix_Y=to_distance_mtx(B))
        test_stat_null, _ = indept_test.test_statistic(
            matrix_X=to_distance_mtx(A_null), matrix_Y=to_distance_mtx(B_sorted))
        
        test_stat_alt_array[i] = test_stat_alt
        test_stat_null_array[i] = test_stat_null
    c1 = np.sort(test_stat_null_array)[math.floor(alpha/2*mc)]
    c2 = np.sort(test_stat_null_array)[math.ceil((1-alpha/2)*mc)]
    power = np.where((test_stat_alt_array > c2) | (test_stat_alt_array < c1))[0].shape[0] / mc
    return power

In [3]:
def pearson_power_two_sided(indept_test, sim_func, mc=500, alpha=0.05,
                  given_blocks=False, blocks=None, **kwargs):
    # power for any test that uses vectorized matrix as samples
    test_stat_null_array = np.zeros(mc)
    test_stat_alt_array = np.zeros(mc)
    for i in range(mc):
        A, B = sim_func(**kwargs)
        if given_blocks:
            block_assignment = blocks
        else:
            block_assignment = estimate_block_assignment(A, B)
        A_null = block_permute(A, block_assignment)
        B_sorted = sort_graph(B, block_assignment)

        test_stat_alt, _ = indept_test.test_statistic(
            matrix_X=triu_no_diag(A), matrix_Y=triu_no_diag(B))
        test_stat_null, _ = indept_test.test_statistic(
            matrix_X=triu_no_diag(A_null), matrix_Y=triu_no_diag(B_sorted))

        test_stat_alt_array[i] = test_stat_alt
        test_stat_null_array[i] = test_stat_null
    c1 = np.sort(test_stat_null_array)[math.floor(alpha/2*mc)]
    c2 = np.sort(test_stat_null_array)[math.ceil((1-alpha/2)*mc)]
    power = np.where((test_stat_alt_array > c2) | (test_stat_alt_array < c1))[0].shape[0] / mc
    return power

In [5]:
def to_distance_mtx_keep_diag(A):
    distance_mtx_A = 1 - (A / np.max(A))
    return distance_mtx_A

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [44]:
k = 2
P1 = sbm_params(a=0.7, b=0.3)
P2 = sbm_params(a=0.7, b=0.3)
n = 100
rho = 0.1
A, B = rho_sbm_marg(rho=rho, AL=P1, BL=P2, k=k, n=n)

In [49]:
test = MGC(compute_distance_matrix=identity)
test.test_statistic(to_distance_mtx_keep_diag(A), to_distance_mtx_keep_diag(B))

(0.7499029344980804,
 {'local_correlation_matrix': array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        , -0.01625815,  0.02395204, ...,  0.14019351,
           0.140064  ,  0.14004405],
         [ 0.        , -0.0242227 ,  0.03539801, ...,  0.2033617 ,
           0.20316139,  0.20313005],
         ...,
         [ 0.        ,  0.13021051,  0.18776879, ...,  0.74431373,
           0.74326054,  0.74313366],
         [ 0.        ,  0.13012574,  0.18763489, ...,  0.74386392,
           0.74280983,  0.74268268],
         [ 0.        ,  0.13010719,  0.18760524, ...,  0.74383598,
           0.74278149,  0.74265425]]),
  'optimal_scale': [35, 34],
  'significant_connected_region': array([[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ...,  True,  True,  True],
    

In [50]:
test.compute_distance_matrix

<function utils.identity(x)>

In [51]:
test = MGC(compute_distance_matrix=identity)
test.test_statistic(to_distance_mtx(A), to_distance_mtx(B))

(0.24773579595093842,
 {'local_correlation_matrix': array([[0.25707148, 0.24709052],
         [0.24793027, 0.2477358 ]]),
  'optimal_scale': [2, 2],
  'significant_connected_region': array([[ True, False],
         [ True, False]])})

In [60]:
%%time
n_arr = np.linspace(10, 100, 10, dtype=int)
rho_arr = np.array([0, 0.1, -0.1])
k = 2
P1 = sbm_params(a=0.7, b=0.3)
P2 = sbm_params(a=0.7, b=0.3)
nmc = 500
test_names = ['pearson', 'dcorr', 'mgc']
power_sbm = {
    'pearson': np.zeros((rho_arr.shape[0], n_arr.shape[0])),
    'dcorr': np.zeros((rho_arr.shape[0], n_arr.shape[0])),
    'mgc': np.zeros((rho_arr.shape[0], n_arr.shape[0]))
}

for name in test_names:
    for i, rho in enumerate(rho_arr):
        for j, n in enumerate(n_arr):
            blocks = np.repeat(np.arange(k), n//k)
            if name == 'pearson':
                test = RVCorr(which_test='pearson')
                test_power = pearson_power_two_sided(test, rho_sbm_marg, given_blocks=True, blocks=blocks,
                                           rho=rho, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            elif name == 'dcorr':
                test = DCorr(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_sbm_marg, given_blocks=True, blocks=blocks,
                                         rho=rho, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            elif name == 'mgc':
                test = MGC(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_sbm_marg, given_blocks=True, blocks=blocks,
                                         rho=rho, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            power_sbm[name][i, j] = test_power
            print('finish {} for rho={}, n={}'.format(name, rho, n))

finish pearson for rho=0.0, n=10
finish pearson for rho=0.0, n=20
finish pearson for rho=0.0, n=30
finish pearson for rho=0.0, n=40
finish pearson for rho=0.0, n=50
finish pearson for rho=0.0, n=60
finish pearson for rho=0.0, n=70
finish pearson for rho=0.0, n=80
finish pearson for rho=0.0, n=90
finish pearson for rho=0.0, n=100
finish pearson for rho=0.1, n=10
finish pearson for rho=0.1, n=20
finish pearson for rho=0.1, n=30
finish pearson for rho=0.1, n=40
finish pearson for rho=0.1, n=50
finish pearson for rho=0.1, n=60
finish pearson for rho=0.1, n=70
finish pearson for rho=0.1, n=80
finish pearson for rho=0.1, n=90
finish pearson for rho=0.1, n=100
finish pearson for rho=-0.1, n=10
finish pearson for rho=-0.1, n=20
finish pearson for rho=-0.1, n=30
finish pearson for rho=-0.1, n=40
finish pearson for rho=-0.1, n=50
finish pearson for rho=-0.1, n=60
finish pearson for rho=-0.1, n=70
finish pearson for rho=-0.1, n=80
finish pearson for rho=-0.1, n=90
finish pearson for rho=-0.1, n=1

In [61]:
power_sbm

{'pearson': array([[0.056, 0.048, 0.064, 0.046, 0.034, 0.032, 0.06 , 0.052, 0.052,
         0.022],
        [0.076, 0.234, 0.418, 0.634, 0.832, 0.956, 0.982, 0.998, 1.   ,
         1.   ],
        [0.1  , 0.236, 0.474, 0.718, 0.81 , 0.938, 0.978, 0.996, 1.   ,
         1.   ]]),
 'dcorr': array([[0.048, 0.058, 0.044, 0.044, 0.046, 0.06 , 0.038, 0.066, 0.06 ,
         0.032],
        [0.054, 0.172, 0.302, 0.592, 0.83 , 0.93 , 0.974, 0.994, 1.   ,
         1.   ],
        [0.072, 0.124, 0.476, 0.624, 0.804, 0.954, 0.992, 0.996, 1.   ,
         1.   ]]),
 'mgc': array([[0.056, 0.062, 0.046, 0.088, 0.038, 0.04 , 0.062, 0.068, 0.08 ,
         0.032],
        [0.06 , 0.19 , 0.294, 0.568, 0.872, 0.954, 0.992, 0.996, 1.   ,
         1.   ],
        [0.108, 0.16 , 0.326, 0.62 , 0.83 , 0.914, 0.99 , 0.994, 1.   ,
         1.   ]])}

In [62]:
with open('../../results/rho_sbm_power_twosided.pkl', 'wb') as f:
    pickle.dump(power_sbm, f)

In [63]:
%%time
n_arr = np.linspace(10, 100, 10, dtype=int)
rho_arr = np.array([0, 0.1, -0.1])
k = 2
P1 = sbm_params(a=0.7, b=0.3)
P2 = sbm_params(a=0.2, b=0.5)
nmc = 500
test_names = ['pearson', 'dcorr', 'mgc']
power_sbm_marg = {
    'pearson': np.zeros((rho_arr.shape[0], n_arr.shape[0])),
    'dcorr': np.zeros((rho_arr.shape[0], n_arr.shape[0])),
    'mgc': np.zeros((rho_arr.shape[0], n_arr.shape[0]))
}

for name in test_names:
    for i, rho in enumerate(rho_arr):
        for j, n in enumerate(n_arr):
            blocks = np.repeat(np.arange(k), n//k)
            if name == 'pearson':
                test = RVCorr(which_test='pearson')
                test_power = pearson_power_two_sided(test, rho_sbm_marg, given_blocks=True, blocks=blocks,
                                           rho=rho, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            elif name == 'dcorr':
                test = DCorr(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_sbm_marg, given_blocks=True, blocks=blocks,
                                         rho=rho, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            elif name == 'mgc':
                test = MGC(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_sbm_marg, given_blocks=True, blocks=blocks,
                                         rho=rho, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            power_sbm_marg[name][i, j] = test_power
            print('finish {} for rho={}, n={}'.format(name, rho, n))

finish pearson for rho=0.0, n=10
finish pearson for rho=0.0, n=20
finish pearson for rho=0.0, n=30
finish pearson for rho=0.0, n=40
finish pearson for rho=0.0, n=50
finish pearson for rho=0.0, n=60
finish pearson for rho=0.0, n=70
finish pearson for rho=0.0, n=80
finish pearson for rho=0.0, n=90
finish pearson for rho=0.0, n=100
finish pearson for rho=0.1, n=10
finish pearson for rho=0.1, n=20
finish pearson for rho=0.1, n=30
finish pearson for rho=0.1, n=40
finish pearson for rho=0.1, n=50
finish pearson for rho=0.1, n=60
finish pearson for rho=0.1, n=70
finish pearson for rho=0.1, n=80
finish pearson for rho=0.1, n=90
finish pearson for rho=0.1, n=100
finish pearson for rho=-0.1, n=10
finish pearson for rho=-0.1, n=20
finish pearson for rho=-0.1, n=30
finish pearson for rho=-0.1, n=40
finish pearson for rho=-0.1, n=50
finish pearson for rho=-0.1, n=60
finish pearson for rho=-0.1, n=70
finish pearson for rho=-0.1, n=80
finish pearson for rho=-0.1, n=90
finish pearson for rho=-0.1, n=1

In [64]:
power_sbm_marg

{'pearson': array([[0.064, 0.036, 0.054, 0.06 , 0.062, 0.046, 0.056, 0.046, 0.046,
         0.06 ],
        [0.046, 0.212, 0.528, 0.71 , 0.826, 0.956, 0.984, 1.   , 1.   ,
         1.   ],
        [0.118, 0.232, 0.418, 0.676, 0.836, 0.964, 0.99 , 0.998, 1.   ,
         1.   ]]),
 'dcorr': array([[0.018, 0.078, 0.058, 0.056, 0.056, 0.042, 0.042, 0.074, 0.054,
         0.07 ],
        [0.068, 0.206, 0.43 , 0.576, 0.8  , 0.938, 0.988, 1.   , 1.   ,
         1.   ],
        [0.08 , 0.314, 0.46 , 0.68 , 0.892, 0.96 , 0.984, 0.994, 1.   ,
         1.   ]]),
 'mgc': array([[0.062, 0.03 , 0.038, 0.054, 0.042, 0.04 , 0.048, 0.06 , 0.038,
         0.046],
        [0.054, 0.194, 0.442, 0.626, 0.822, 0.934, 0.984, 0.998, 1.   ,
         1.   ],
        [0.074, 0.212, 0.4  , 0.628, 0.806, 0.952, 0.994, 0.998, 1.   ,
         1.   ]])}

In [65]:
with open('../../results/rho_sbm_marg_power_twosided.pkl', 'wb') as f:
    pickle.dump(power_sbm_marg, f)

In [66]:
n_arr = np.linspace(10, 100, 10, dtype=int)
rho_arr = np.array([0, 0.1, -0.1])
p = 0.5
q = 0.5
nmc = 500
k = 1
test_names = ['pearson', 'dcorr', 'mgc']
power_er = {
    'pearson': np.zeros((rho_arr.shape[0], n_arr.shape[0])),
    'dcorr': np.zeros((rho_arr.shape[0], n_arr.shape[0])),
    'mgc': np.zeros((rho_arr.shape[0], n_arr.shape[0]))
}


for name in test_names:
    for i, rho in enumerate(rho_arr):
        for j, n in enumerate(n_arr):
            blocks = np.repeat(np.arange(k), n//k)
            if name == 'pearson':
                test = RVCorr(which_test='pearson')
                test_power = pearson_power_two_sided(test, rho_ER_marg, given_blocks=True, blocks=blocks,
                                           rho=rho, p=p, q=q, n=n, mc=nmc)
            elif name == 'dcorr':
                test = DCorr(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_ER_marg, given_blocks=True, blocks=blocks,
                                           rho=rho, p=p, q=q, n=n, mc=nmc)
            elif name == 'mgc':
                test = MGC(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_ER_marg, given_blocks=True, blocks=blocks,
                                           rho=rho, p=p, q=q, n=n, mc=nmc)
            power_er[name][i, j] = test_power
            print('finish {} for rho={}, n={}'.format(name, rho, n))

finish pearson for rho=0.0, n=10
finish pearson for rho=0.0, n=20
finish pearson for rho=0.0, n=30
finish pearson for rho=0.0, n=40
finish pearson for rho=0.0, n=50
finish pearson for rho=0.0, n=60
finish pearson for rho=0.0, n=70
finish pearson for rho=0.0, n=80
finish pearson for rho=0.0, n=90
finish pearson for rho=0.0, n=100
finish pearson for rho=0.1, n=10
finish pearson for rho=0.1, n=20
finish pearson for rho=0.1, n=30
finish pearson for rho=0.1, n=40
finish pearson for rho=0.1, n=50
finish pearson for rho=0.1, n=60
finish pearson for rho=0.1, n=70
finish pearson for rho=0.1, n=80
finish pearson for rho=0.1, n=90
finish pearson for rho=0.1, n=100
finish pearson for rho=-0.1, n=10
finish pearson for rho=-0.1, n=20
finish pearson for rho=-0.1, n=30
finish pearson for rho=-0.1, n=40
finish pearson for rho=-0.1, n=50
finish pearson for rho=-0.1, n=60
finish pearson for rho=-0.1, n=70
finish pearson for rho=-0.1, n=80
finish pearson for rho=-0.1, n=90
finish pearson for rho=-0.1, n=1

In [69]:
power_er

{'pearson': array([[0.03 , 0.038, 0.058, 0.054, 0.042, 0.032, 0.1  , 0.058, 0.034,
         0.042],
        [0.104, 0.296, 0.528, 0.744, 0.948, 0.982, 1.   , 1.   , 1.   ,
         1.   ],
        [0.084, 0.238, 0.5  , 0.792, 0.9  , 0.986, 1.   , 1.   , 1.   ,
         1.   ]]),
 'dcorr': array([[0.054, 0.034, 0.06 , 0.042, 0.076, 0.062, 0.068, 0.072, 0.034,
         0.048],
        [0.066, 0.222, 0.548, 0.724, 0.926, 0.98 , 1.   , 1.   , 1.   ,
         1.   ],
        [0.078, 0.302, 0.506, 0.73 , 0.932, 0.996, 1.   , 1.   , 1.   ,
         1.   ]]),
 'mgc': array([[0.04 , 0.082, 0.054, 0.042, 0.014, 0.066, 0.058, 0.05 , 0.038,
         0.036],
        [0.072, 0.224, 0.508, 0.774, 0.89 , 0.972, 1.   , 1.   , 0.998,
         1.   ],
        [0.062, 0.254, 0.552, 0.812, 0.942, 0.984, 0.996, 1.   , 1.   ,
         1.   ]])}

In [70]:
with open('../../results/rho_er_power_twosided.pkl', 'wb') as f:
    pickle.dump(power_er, f)

In [72]:
n_arr = np.linspace(10, 100, 10, dtype=int)
rho_arr = np.array([0, 0.1, -0.1])
p = 0.7
q = 0.2
nmc = 500
k = 1
test_names = ['pearson', 'dcorr', 'mgc']
power_er_marg = {
    'pearson': np.zeros((rho_arr.shape[0], n_arr.shape[0])),
    'dcorr': np.zeros((rho_arr.shape[0], n_arr.shape[0])),
    'mgc': np.zeros((rho_arr.shape[0], n_arr.shape[0]))
}


for name in test_names:
    for i, rho in enumerate(rho_arr):
        for j, n in enumerate(n_arr):
            blocks = np.repeat(np.arange(k), n//k)
            if name == 'pearson':
                test = RVCorr(which_test='pearson')
                test_power = pearson_power_two_sided(test, rho_ER_marg, given_blocks=True, blocks=blocks,
                                           rho=rho, p=p, q=q, n=n, mc=nmc)
            elif name == 'dcorr':
                test = DCorr(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_ER_marg, given_blocks=True, blocks=blocks,
                                           rho=rho, p=p, q=q, n=n, mc=nmc)
            elif name == 'mgc':
                test = MGC(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_ER_marg, given_blocks=True, blocks=blocks,
                                           rho=rho, p=p, q=q, n=n, mc=nmc)
            power_er_marg[name][i, j] = test_power
            print('finish {} for rho={}, n={}'.format(name, rho, n))

finish pearson for rho=0.0, n=10
finish pearson for rho=0.0, n=20
finish pearson for rho=0.0, n=30
finish pearson for rho=0.0, n=40
finish pearson for rho=0.0, n=50
finish pearson for rho=0.0, n=60
finish pearson for rho=0.0, n=70
finish pearson for rho=0.0, n=80
finish pearson for rho=0.0, n=90
finish pearson for rho=0.0, n=100
finish pearson for rho=0.1, n=10
finish pearson for rho=0.1, n=20
finish pearson for rho=0.1, n=30
finish pearson for rho=0.1, n=40
finish pearson for rho=0.1, n=50
finish pearson for rho=0.1, n=60
finish pearson for rho=0.1, n=70
finish pearson for rho=0.1, n=80
finish pearson for rho=0.1, n=90
finish pearson for rho=0.1, n=100
finish pearson for rho=-0.1, n=10
finish pearson for rho=-0.1, n=20
finish pearson for rho=-0.1, n=30
finish pearson for rho=-0.1, n=40
finish pearson for rho=-0.1, n=50
finish pearson for rho=-0.1, n=60
finish pearson for rho=-0.1, n=70
finish pearson for rho=-0.1, n=80
finish pearson for rho=-0.1, n=90
finish pearson for rho=-0.1, n=1

In [73]:
power_er_marg

{'pearson': array([[0.06 , 0.048, 0.042, 0.048, 0.072, 0.03 , 0.038, 0.04 , 0.038,
         0.046],
        [0.08 , 0.252, 0.492, 0.82 , 0.944, 0.98 , 0.998, 1.   , 1.   ,
         1.   ],
        [0.068, 0.272, 0.556, 0.782, 0.942, 0.992, 1.   , 0.998, 1.   ,
         1.   ]]),
 'dcorr': array([[0.048, 0.024, 0.048, 0.046, 0.06 , 0.03 , 0.066, 0.02 , 0.06 ,
         0.02 ],
        [0.062, 0.21 , 0.586, 0.794, 0.932, 0.996, 0.998, 0.998, 1.   ,
         1.   ],
        [0.034, 0.352, 0.444, 0.792, 0.91 , 0.98 , 0.996, 1.   , 1.   ,
         1.   ]]),
 'mgc': array([[0.042, 0.06 , 0.038, 0.042, 0.042, 0.056, 0.026, 0.042, 0.038,
         0.062],
        [0.064, 0.29 , 0.572, 0.824, 0.926, 0.992, 1.   , 1.   , 1.   ,
         1.   ],
        [0.104, 0.274, 0.574, 0.804, 0.918, 0.98 , 0.998, 1.   , 1.   ,
         1.   ]])}

In [76]:
with open('../../results/rho_er_marg_power_twosided.pkl', 'wb') as f:
    pickle.dump(power_er_marg, f)

In [5]:
%%time
n_arr = np.linspace(10, 100, 10, dtype=int)
rho_arr = np.array([0, 0.1, -0.1])
k = 2
P1 = sbm_params(a=0.7, b=0.3)
P2 = sbm_params(a=0.2, b=0.5)
nmc = 500
test_names = ['pearson', 'dcorr', 'mgc']
power_sbm_est_block = {
    'pearson': np.zeros((rho_arr.shape[0], n_arr.shape[0])),
    'dcorr': np.zeros((rho_arr.shape[0], n_arr.shape[0])),
    'mgc': np.zeros((rho_arr.shape[0], n_arr.shape[0]))
}

for name in test_names:
    for i, rho in enumerate(rho_arr):
        for j, n in enumerate(n_arr):
            if name == 'pearson':
                test = RVCorr(which_test='pearson')
                test_power = pearson_power_two_sided(test, rho_sbm_marg, rho=rho, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            elif name == 'dcorr':
                test = DCorr(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_sbm_marg, rho=rho, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            elif name == 'mgc':
                test = MGC(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_sbm_marg, rho=rho, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            power_sbm_est_block[name][i, j] = test_power
            print('finish {} for rho={}, n={}'.format(name, rho, n))

finish pearson for rho=0.0, n=10
finish pearson for rho=0.0, n=20
finish pearson for rho=0.0, n=30
finish pearson for rho=0.0, n=40
finish pearson for rho=0.0, n=50
finish pearson for rho=0.0, n=60
finish pearson for rho=0.0, n=70
finish pearson for rho=0.0, n=80
finish pearson for rho=0.0, n=90
finish pearson for rho=0.0, n=100
finish pearson for rho=0.1, n=10
finish pearson for rho=0.1, n=20
finish pearson for rho=0.1, n=30
finish pearson for rho=0.1, n=40
finish pearson for rho=0.1, n=50
finish pearson for rho=0.1, n=60
finish pearson for rho=0.1, n=70
finish pearson for rho=0.1, n=80
finish pearson for rho=0.1, n=90
finish pearson for rho=0.1, n=100
finish pearson for rho=-0.1, n=10
finish pearson for rho=-0.1, n=20
finish pearson for rho=-0.1, n=30
finish pearson for rho=-0.1, n=40
finish pearson for rho=-0.1, n=50
finish pearson for rho=-0.1, n=60
finish pearson for rho=-0.1, n=70
finish pearson for rho=-0.1, n=80
finish pearson for rho=-0.1, n=90
finish pearson for rho=-0.1, n=1

/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1704: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)


finish mgc for rho=0.0, n=10
finish mgc for rho=0.0, n=20
finish mgc for rho=0.0, n=30
finish mgc for rho=0.0, n=40
finish mgc for rho=0.0, n=50
finish mgc for rho=0.0, n=60
finish mgc for rho=0.0, n=70
finish mgc for rho=0.0, n=80
finish mgc for rho=0.0, n=90
finish mgc for rho=0.0, n=100


/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1704: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)


finish mgc for rho=0.1, n=10
finish mgc for rho=0.1, n=20
finish mgc for rho=0.1, n=30
finish mgc for rho=0.1, n=40
finish mgc for rho=0.1, n=50
finish mgc for rho=0.1, n=60
finish mgc for rho=0.1, n=70
finish mgc for rho=0.1, n=80
finish mgc for rho=0.1, n=90
finish mgc for rho=0.1, n=100


/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1704: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)


finish mgc for rho=-0.1, n=10
finish mgc for rho=-0.1, n=20
finish mgc for rho=-0.1, n=30
finish mgc for rho=-0.1, n=40
finish mgc for rho=-0.1, n=50
finish mgc for rho=-0.1, n=60
finish mgc for rho=-0.1, n=70
finish mgc for rho=-0.1, n=80
finish mgc for rho=-0.1, n=90
finish mgc for rho=-0.1, n=100
CPU times: user 2h 26min 53s, sys: 8min 49s, total: 2h 35min 42s
Wall time: 1h 33min 3s


In [9]:
power_sbm_est_block

{'pearson': array([[0.04 , 0.036, 0.02 , 0.036, 0.05 , 0.068, 0.044, 0.056, 0.058,
         0.04 ],
        [0.026, 0.022, 0.064, 0.7  , 0.89 , 0.932, 0.996, 0.994, 1.   ,
         1.   ],
        [0.046, 0.184, 0.438, 0.662, 0.862, 0.954, 0.984, 0.998, 1.   ,
         1.   ]]),
 'dcorr': array([[0.024, 0.016, 0.032, 0.042, 0.04 , 0.04 , 0.056, 0.046, 0.04 ,
         0.054],
        [0.04 , 0.004, 0.094, 0.574, 0.858, 0.956, 0.984, 0.998, 1.   ,
         1.   ],
        [0.032, 0.126, 0.39 , 0.71 , 0.812, 0.958, 0.982, 0.996, 1.   ,
         1.   ]]),
 'mgc': array([[0.032, 0.02 , 0.03 , 0.05 , 0.072, 0.036, 0.078, 0.052, 0.056,
         0.05 ],
        [0.024, 0.032, 0.044, 0.68 , 0.874, 0.956, 0.99 , 0.994, 0.998,
         1.   ],
        [0.06 , 0.114, 0.462, 0.7  , 0.804, 0.958, 0.994, 1.   , 1.   ,
         1.   ]])}

In [10]:
with open('../../results/rho_sbm_marg_est_block_power_twosided.pkl', 'wb') as f:
    pickle.dump(power_sbm_est_block, f)

In [11]:
%%time
n_arr = np.linspace(10, 100, 10, dtype=int)
rho_arr = np.array([0, 0.1, -0.1])
P1 = sbm_params(a=0.7, b=0.3)
P2 = sbm_params(a=0.2, b=0.5)
nmc = 500
test_names = ['pearson', 'dcorr', 'mgc']
power_sbm_diff_block = {
    'pearson': np.zeros((rho_arr.shape[0], n_arr.shape[0])),
    'dcorr': np.zeros((rho_arr.shape[0], n_arr.shape[0])),
    'mgc': np.zeros((rho_arr.shape[0], n_arr.shape[0]))
}

for name in test_names:
    for i, rho in enumerate(rho_arr):
        for j, n in enumerate(n_arr):
            k = [int(0.7*n), n-int(0.7*n)]
            if name == 'pearson':
                test = RVCorr(which_test='pearson')
                test_power = pearson_power_two_sided(test, rho_sbm_diff_block, rho=rho, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            elif name == 'dcorr':
                test = DCorr(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_sbm_diff_block, rho=rho, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            elif name == 'mgc':
                test = MGC(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_sbm_diff_block, rho=rho, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            power_sbm_diff_block[name][i, j] = test_power
            print('finish {} for rho={}, n={}'.format(name, rho, n))

/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return_n_iter=True)


finish pearson for rho=0.0, n=10
finish pearson for rho=0.0, n=20
finish pearson for rho=0.0, n=30
finish pearson for rho=0.0, n=40
finish pearson for rho=0.0, n=50
finish pearson for rho=0.0, n=60
finish pearson for rho=0.0, n=70
finish pearson for rho=0.0, n=80
finish pearson for rho=0.0, n=90
finish pearson for rho=0.0, n=100
finish pearson for rho=0.1, n=10
finish pearson for rho=0.1, n=20
finish pearson for rho=0.1, n=30
finish pearson for rho=0.1, n=40
finish pearson for rho=0.1, n=50
finish pearson for rho=0.1, n=60
finish pearson for rho=0.1, n=70
finish pearson for rho=0.1, n=80
finish pearson for rho=0.1, n=90
finish pearson for rho=0.1, n=100


/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1704: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return_n_iter=True)


finish pearson for rho=-0.1, n=10
finish pearson for rho=-0.1, n=20
finish pearson for rho=-0.1, n=30
finish pearson for rho=-0.1, n=40
finish pearson for rho=-0.1, n=50
finish pearson for rho=-0.1, n=60
finish pearson for rho=-0.1, n=70
finish pearson for rho=-0.1, n=80
finish pearson for rho=-0.1, n=90
finish pearson for rho=-0.1, n=100


/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1704: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1704: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/

...

finish mgc for rho=0.0, n=10
finish mgc for rho=0.0, n=20
finish mgc for rho=0.0, n=30
finish mgc for rho=0.0, n=40
finish mgc for rho=0.0, n=50
finish mgc for rho=0.0, n=60
finish mgc for rho=0.0, n=70
finish mgc for rho=0.0, n=80
finish mgc for rho=0.0, n=90
finish mgc for rho=0.0, n=100


/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return_n_iter=True)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1704: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)
/Users/jxiong/Documents/Projects/graph_

finish mgc for rho=0.1, n=10
finish mgc for rho=0.1, n=20
finish mgc for rho=0.1, n=30
finish mgc for rho=0.1, n=40
finish mgc for rho=0.1, n=50
finish mgc for rho=0.1, n=60
finish mgc for rho=0.1, n=70
finish mgc for rho=0.1, n=80
finish mgc for rho=0.1, n=90
finish mgc for rho=0.1, n=100


/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1704: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
/Users/jxiong/Documents/Projects/graph_independence_test/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)


finish mgc for rho=-0.1, n=10
finish mgc for rho=-0.1, n=20
finish mgc for rho=-0.1, n=30
finish mgc for rho=-0.1, n=40
finish mgc for rho=-0.1, n=50
finish mgc for rho=-0.1, n=60
finish mgc for rho=-0.1, n=70
finish mgc for rho=-0.1, n=80
finish mgc for rho=-0.1, n=90
finish mgc for rho=-0.1, n=100
CPU times: user 2h 21min 57s, sys: 8min 31s, total: 2h 30min 28s
Wall time: 1h 26min 30s


In [12]:
power_sbm_diff_block

{'pearson': array([[0.034, 0.026, 0.048, 0.008, 0.026, 0.042, 0.05 , 0.076, 0.056,
         0.054],
        [0.038, 0.01 , 0.004, 0.614, 0.78 , 0.954, 0.994, 1.   , 0.998,
         1.   ],
        [0.044, 0.18 , 0.438, 0.724, 0.834, 0.978, 0.994, 0.996, 1.   ,
         1.   ]]),
 'dcorr': array([[0.054, 0.044, 0.03 , 0.014, 0.04 , 0.04 , 0.068, 0.062, 0.054,
         0.064],
        [0.034, 0.024, 0.046, 0.564, 0.852, 0.958, 0.986, 1.   , 1.   ,
         1.   ],
        [0.05 , 0.13 , 0.424, 0.704, 0.892, 0.97 , 0.994, 1.   , 1.   ,
         1.   ]]),
 'mgc': array([[0.03 , 0.022, 0.024, 0.016, 0.06 , 0.046, 0.052, 0.05 , 0.048,
         0.056],
        [0.046, 0.042, 0.074, 0.322, 0.878, 0.946, 0.994, 1.   , 0.998,
         1.   ],
        [0.046, 0.14 , 0.402, 0.652, 0.838, 0.954, 0.992, 0.996, 1.   ,
         1.   ]])}

In [13]:
with open('../../results/rho_sbm_diff_block_power_twosided.pkl', 'wb') as f:
    pickle.dump(power_sbm_diff_block, f)